The goal of the project is to use Machine Learning algorithms to track Functional JavaScript codes.
The dataset given for the project contains JavaScript source codes: - Folder 1 contains JS source
codes used in tracking user activity while surfing - Folder 2 contains useful JS source codes used
to enable the functioning of web applications. The goal of the project is to process the data given,
extract features from it and use it to train listed clasifier models. The models should then be
tested: see how they perform when it comes to predicting whether fed in data contains tracking JS
or functional JS codes.

In [1]:
from __future__ import division
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from operator import itemgetter
from sklearn.metrics import classification_report
import csv, os, codecs, fnmatch
import numpy as np
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import train_test_split
import pandas as pd


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
usefulJSpaths = [] 
usefulJSfiles = [] 
rusefulJSfiles = [] 
n_features = 190

gmv = 0.001 # 0.001 # 0.05 # 0.1 # 0.5 # 0.1, 0.05, 0.01, 0.005, 0.001
nuv = 0.001 # 0.1, 0.05, 0.01, 0.005, 0.001

for root, dirnames, filenames, in os.walk('functionalJS'):
    for filename in fnmatch.filter(filenames, "*.js"):
        usefulJSfiles.append(filename)
        usefulJSpaths.append(os.path.join(root, filename))
        try:
            with codecs.open(os.path.join(root, filename), encoding="ISO-8859-1") as f: 
                rusefulJSfiles.append(str(f.read())) 
            except IOError as exc:
            if exc.errno != errno.EISDIR: 
                raise 

uselessJSpaths = [] 
uselessJSfiles = [] 
ruselessJSfiles = []


for ulessroot, ulessdirnames, ulessfilenames, in os.walk('trackingJS'):
    for filename in fnmatch.filter(ulessfilenames, "*.js"):
        uselessJSfiles.append(filename)
        uselessJSpaths.append(os.path.join(ulessroot, filename))
        try:
            with codecs.open(os.path.join(ulessroot, filename), encoding="ISO-8859-1") as f: 
                ruselessJSfiles.append(str(f.read())) 
            except IOError as exc:
            if exc.errno != errno.EISDIR: 
                raise 


In [30]:
# Converting data to dataframe and encoding. 
# .....1 for functional JS code (useful JS code)
# .....0 for tracking JS code (useless JS code)

df_functional = pd.DataFrame(rusefulJSfiles, columns = ['code'])
df_functional['label'] = 1
df_tracking = pd.DataFrame(ruselessJSfiles, columns = ['code'])
df_tracking['label'] = 0

In [31]:
# preview
df_functional.head(5)

,code,label
0,Type.registerNamespace('AjaxControlToolkit');\...,1
1,"function getQueryValue(n, t) {\n var r = ne...",1
2,"(function(e, f) {\n var a = f.location.hash...",1
3,/*!\n * jQuery Form Plugin\n * version: 3.51.0...,1
4,"ServerJSQueue.add(function(){requireLazy([""Boo...",1


In [191]:
#preview
df_tracking.head(5)

,code,label
0,var d16gCollapse=true;frameElement.parentNode....,0
1,"(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObj...",0
2,"var urchinTracker=function(){},_gaq={push:func...",0
3,"(function () {\n var scheme = ((""https:"" ==...",0
4,"window.NREUM||(NREUM={});NREUM.info={""beacon"":...",0


In [192]:
# concat the two dataframes together
df = pd.concat([df_functional, df_tracking]) 
df.tail(5)

,code,label
1484,"/* 30,79,6 2013-07-18 18:23:53 */\n\nfunction ...",0
1485,(function() {\n var bds = window.bds || {};...,0
1486,\n (function($){\n var winWidth = $(...,0
1487,var AF_initDataKeys = []\n; var AF_dataService...,0
1488,"_satellite.pushAsyncScript(function(event, tar...",0


In [193]:
X = df['code'].values
y = df['label'].values

#### Feature Extraction Using TF-IDF Vectorization

In [194]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

In [195]:
X.shape

(2612, 419068)

#### Feature Reduction Using SelectKBest

In [197]:
from sklearn.feature_selection import SelectKBest

X = SelectKBest(k=50000).fit_transform(X, df['label'])

In [198]:
X.shape

(2612, 50000)

#### Splitting data

In [199]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=40)

### Modeling

##### Baseline SVM (SVC)

In [200]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train)
y_pred= clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83       445
           1       0.79      0.74      0.76       339

    accuracy                           0.80       784
   macro avg       0.80      0.79      0.80       784
weighted avg       0.80      0.80      0.80       784



##### OneClassSVM

In [77]:
from sklearn.svm import OneClassSVM

RBF Kernel

In [175]:
clf = OneClassSVM(kernel='rbf',degree=3 , gamma='scale', coef0=0.05, tol=0.001 , nu=.3)
clf.fit(X_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00       445
           1       0.37      0.48      0.42       339

    accuracy                           0.21       784
   macro avg       0.12      0.16      0.14       784
weighted avg       0.16      0.21      0.18       784



Linear Kernel

In [164]:
clf = OneClassSVM(kernel='linear',degree=1 , gamma='scale', coef0=10, tol=0.001 , nu=.3)
clf.fit(X_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00       445
           1       0.40      0.59      0.48       339

    accuracy                           0.26       784
   macro avg       0.13      0.20      0.16       784
weighted avg       0.17      0.26      0.21       784



Poly Kernel

In [180]:
clf = OneClassSVM(kernel='poly',degree=3 , gamma='auto', coef0=10, tol=0.001 , nu=.3 )
clf.fit(X_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00       445
           1       0.42      0.94      0.58       339

    accuracy                           0.40       784
   macro avg       0.14      0.31      0.19       784
weighted avg       0.18      0.40      0.25       784



Sigmoid Kernel

In [179]:
clf = OneClassSVM(kernel='sigmoid',degree=3 , gamma=.12, coef0=0.9, tol=0.001 , nu=.3 )
clf.fit(X_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00       445
           1       0.40      0.59      0.47       339

    accuracy                           0.26       784
   macro avg       0.13      0.20      0.16       784
weighted avg       0.17      0.26      0.21       784

